In [5]:
import pandas as pd
import re

In [6]:
data = pd.read_csv("NHoodNameCentroids.csv")

In [7]:
data.head()

,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough
0,POINT (-73.8472005205491 40.89470517661004),1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx
1,POINT (-73.82993910812405 40.87429419303015),2,Co-op City,2,Co-op,City,NaN,0,Bronx
2,POINT (-73.82780644716419 40.88755567735082),3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx
3,POINT (-73.90564259591689 40.895437426903875),4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx
4,POINT (-73.91258546108577 40.89083449389134),5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx


In [8]:
data['latitude'] = '0'
data['longitude'] = '0'
data.dtypes

the_geom     object
OBJECTID      int64
Name         object
Stacked       int64
AnnoLine1    object
AnnoLine2    object
AnnoLine3    object
AnnoAngle     int64
Borough      object
latitude     object
longitude    object
dtype: object

In [9]:
for index, row in data.iterrows():
    lat_long = re.search('POINT \((.+?)\)',  row['the_geom'])
    if lat_long:
        data['latitude'][index] = lat_long.group(1).split(' ')[1]
        data['longitude'][index] = lat_long.group(1).split(' ')[0]
data.head()

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,the_geom,OBJECTID,Name,Stacked,AnnoLine1,AnnoLine2,AnnoLine3,AnnoAngle,Borough,latitude,longitude
0,POINT (-73.8472005205491 40.89470517661004),1,Wakefield,1,Wakefield,NaN,NaN,0,Bronx,40.89470517661004,-73.8472005205491
1,POINT (-73.82993910812405 40.87429419303015),2,Co-op City,2,Co-op,City,NaN,0,Bronx,40.87429419303015,-73.82993910812405
2,POINT (-73.82780644716419 40.88755567735082),3,Eastchester,1,Eastchester,NaN,NaN,0,Bronx,40.88755567735082,-73.82780644716419
3,POINT (-73.90564259591689 40.895437426903875),4,Fieldston,1,Fieldston,NaN,NaN,0,Bronx,40.895437426903875,-73.90564259591689
4,POINT (-73.91258546108577 40.89083449389134),5,Riverdale,1,Riverdale,NaN,NaN,0,Bronx,40.89083449389134,-73.91258546108577


In [10]:
manhattan_data = data[data['Borough'] == 'Manhattan'].reset_index(drop=True)
drop_columns = ['the_geom', 'OBJECTID', 'Stacked', 'AnnoLine1', 'AnnoLine2', 'AnnoLine3', 'AnnoAngle']
for col in drop_columns:
    del manhattan_data[col]

In [63]:
manhattan_data.head()

,Name,Borough,latitude,longitude
0,Marble Hill,Manhattan,40.87655077879968,-73.91065965862988
1,Chinatown,Manhattan,40.715618422314364,-73.99427936255985
2,Washington Heights,Manhattan,40.85190252555309,-73.93690027985242
3,Inwood,Manhattan,40.86768396449919,-73.92121042203905
4,Hamilton Heights,Manhattan,40.82360428481197,-73.94968791883373


In [12]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [13]:
from geopy.geocoders import Nominatim

In [14]:
address = 'Manhattan, NY'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Manhattan are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Manhattan are 40.7896239, -73.9598939.


In [24]:
CLIENT_ID = 'C1UYVLF4RL5AWWT22ISOZIFGKGELWO3MKB345KNQVCAPKYX3'
CLIENT_SECRET = 'HPP5M3BSQE3DGNEPI4HHIK5LHNWAV0FIW23ZMDQWBTBM5M0H'
LIMIT = 10000
RADIUS = 500
VERSION = '20180605'

In [25]:
import requests

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
manhattan_venues = getNearbyVenues(names=manhattan_data['Name'],
                                   latitudes=manhattan_data['latitude'],
                                   longitudes=manhattan_data['longitude']
                                  )

Marble Hill
Chinatown
Washington Heights
Inwood
Hamilton Heights
Manhattanville
Central Harlem
East Harlem
Upper East Side
Yorkville
Lenox Hill
Roosevelt Island
Upper West Side
Lincoln Square
Clinton
Midtown
Murray Hill
Chelsea
Greenwich Village
East Village
Lower East Side
Tribeca
Little Italy
Soho
West Village
Manhattan Valley
Morningside Heights
Gramercy
Battery Park City
Financial District
Carnegie Hill
Noho
Civic Center
Midtown South
Sutton Place
Turtle Bay
Tudor City
Stuyvesant Town
Flatiron


In [28]:
manhattan_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.87655077879968,-73.91065965862988,Arturo's,40.874412,-73.910271,Pizza Place
1,Marble Hill,40.87655077879968,-73.91065965862988,Bikram Yoga,40.876844,-73.906204,Yoga Studio
2,Marble Hill,40.87655077879968,-73.91065965862988,Tibbett Diner,40.880404,-73.908937,Diner
3,Marble Hill,40.87655077879968,-73.91065965862988,Starbucks,40.877531,-73.905582,Coffee Shop
4,Marble Hill,40.87655077879968,-73.91065965862988,Dunkin',40.877136,-73.906666,Donut Shop


In [38]:
# Examiing the venue category 
venue_categories = list(set(manhattan_venues['Venue Category']))
print(venue_categories)


['Resort', 'Rest Area', 'Soup Place', 'Shipping Store', 'English Restaurant', 'Fast Food Restaurant', 'Moving Target', 'Israeli Restaurant', 'Vegetarian / Vegan Restaurant', 'Bus Line', 'Food Truck', 'Bike Shop', 'Trail', 'Ski Shop', 'Bike Trail', 'Medical Center', 'Nightclub', 'Music Venue', 'Skating Rink', 'Fountain', 'Wine Bar', 'Boat or Ferry', 'African Restaurant', 'Used Bookstore', 'Dumpling Restaurant', 'Garden Center', 'Bar', 'Non-Profit', 'Health Food Store', 'Rock Club', 'Health & Beauty Service', 'Coworking Space', 'Outdoors & Recreation', 'Bank', 'Burger Joint', 'Arcade', 'Jewish Restaurant', 'Vietnamese Restaurant', 'Breakfast Spot', 'Food & Drink Shop', 'Beer Store', 'Golf Course', 'Yoga Studio', 'Italian Restaurant', 'Playground', 'Miscellaneous Shop', 'Antique Shop', 'Gymnastics Gym', 'Bike Rental / Bike Share', 'Grocery Store', 'Movie Theater', 'Hobby Shop', 'Dry Cleaner', 'Sculpture Garden', 'Office', 'Ramen Restaurant', 'Rock Climbing Spot', 'Soccer Field', 'Southern

In [40]:
sample_categories = ['school', 'college', 'university', 'tea', 'coffee', 'office', 'center', 'cafeteria', 'café']
filtered_categories = []
for venue in venue_categories:
    venue_lower = venue.lower()
    for sample_venue in sample_categories:
        if sample_venue in venue_lower:
            filtered_categories.append(venue)
print(filtered_categories)

['Medical Center', 'Garden Center', 'Office', 'Tourist Information Center', 'College Bookstore', 'Cafeteria', 'Cultural Center', 'School', 'Weight Loss Center', "Doctor's Office", 'Spiritual Center', 'Gym / Fitness Center', 'Steakhouse', 'Paper / Office Supplies Store', 'Coffee Shop', 'Café', 'College Arts Building', 'College Theater', 'Music School', 'Pet Café', 'Tea Room', 'Community Center', 'Cooking School', 'College Academic Building', 'College Cafeteria', 'College Cafeteria', 'High School', 'Recreation Center', 'Bubble Tea Shop']


In [42]:
#Filerring out the relevant categories
relevant_categories = ['Office', 'College Bookstore', 'Cafeteria', 'School', 'Coffee Shop', 'Café', 'College Arts Building', 'College Theater',  'Music School', 'Tea Room', 'Cooking School','College Academic Building', 'College Cafeteria', 'College Cafeteria', 'High School', 'Bubble Tea Shop'] 

In [47]:
manhattan_venues = manhattan_venues[manhattan_venues['Venue Category'].isin(relevant_categories)]
type(manhattan_venues)

pandas.core.frame.DataFrame